In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용

Output()

In [16]:
''' 파일 불러오기 '''
df = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2020.csv", encoding='UTF-8')
print(df.shape, df.columns)
df

(164993, 3) Index(['sentence', 'predicted_sentiment', 'org_idx'], dtype='object')


,sentence,predicted_sentiment,org_idx
0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,부정,0
1,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,긍정,0
2,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,긍정,0
3,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,긍정,0
4,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,긍정,0
...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,긍정,5787
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,긍정,5787
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,긍정,5787
164991,2020년 1월 7일 하루 종일 비가 내린다.,긍정,5787


In [17]:
for emo in ['긍정','부정','중립']:

    ''' 형태소 분석 '''
    nouns = []
    verbs = []
    adjs = []

    df_class = df[df['predicted_sentiment'] == "%s"%emo]
    corpus = df_class['sentence'].to_list()

    for sent in tqdm(range(len(corpus))):
        # 문장에서 형태소/품사 추출
        try:
            a = okt.pos(corpus[sent], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
            for x, y in a:
                # 품사가 명사면 명사 리스트에 단어 추가
                if y == 'Noun':
                    nouns.append(x)
                # 품사가 동사면 동사 리스트에 단어 추가
                elif y == 'Verb':
                    verbs.append(x)
                # 품사가 형용사면 형용사 리스트에 단어 추가
                elif y == 'Adjective':
                    adjs.append(x)
        except:
            pass

    morph_dic = {'명사':nouns, '동사':verbs, '형용사':adjs}
    for mor in ['명사','동사','형용사']:
        
        # 명사 리스트에서 명사 빈도 리스트 생성
        morph_cnt = Counter(morph_dic[mor]).most_common()

        morph = []
        morph_freq = []
        for key, val in morph_cnt:
            morph.append(key)
            morph_freq.append(val)

        # 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
        morph_df = pd.DataFrame({'morph':morph, 'morph_freq':morph_freq})
        morph_df.to_csv("2020_감정예측_%s_%s.csv"%(emo, mor), index=False, encoding='UTF-8')

100%|██████████| 124/124 [00:01<00:00, 85.37it/s] 


In [18]:
import plotly.express as px

# unique 값들의 개수를 계산
sentiment_counts = df['predicted_sentiment'].value_counts()


# 막대그래프로 시각화
fig = px.bar(sentiment_counts, x=sentiment_counts.index, y=sentiment_counts.values,
             labels={'x': 'Sentiment', 'y': 'Count'},
             title='Sentiment Distribution 2020',
             color=sentiment_counts.index,
             text=sentiment_counts.values)  # 개수를 막대에 표시하기 위해 text 속성에 sentiment_counts.values 할당

# 개수를 막대 위에 표시하는 옵션 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# y축 범위 설정 (기존 최대값의 10% 더 큰 값으로 설정)
y_axis_range = sentiment_counts.max() * 1.1
fig.update_layout(yaxis_range=[0, y_axis_range])

# y축 값의 형식 설정 (정수 형식으로 표시)
fig.update_layout(yaxis_tickformat=',d')

# 글꼴 크기 설정
fig.update_layout(
    font=dict(
        family="Arial, bold",
        size=13
    )
)

# x축 레이블 설정
fig.update_xaxes(title_text='Label')

# 제목을 가운데에 위치시키기
fig.update_layout(title_x=0.5)

# 그래프 출력
fig.show()